<a href="https://colab.research.google.com/github/m-mehabadi/grad-maker/blob/main/_notebooks/Testing_GradientMaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

http://www.cs.cmu.edu/~pradeepr/convexopt/Lecture_Slides/dual-ascent.pdf

In [27]:
import numpy as np
# import torch
import matplotlib.pyplot as plt

In [2]:
# dim=2000000
# a = 100*np.random.randn(4, dim) + 1123
# g = np.random.randn(dim, 1)
# b = (a-np.mean(a, axis=1)[:, np.newaxis])/np.std(a, axis=1)[:, np.newaxis]
# np.mean(b@g/dim)

In [28]:
def gradient_maker(domain_grads, epsilon=0.5, alpha=0.01, eff=0.1):
    # def norm(vec):
    #     return np.sqrt(np.sum(vec**2))
    def log():
        print(f"Iter={iter}, Condition={np.min(dgr@g/dim)}")

    dgr = np.copy(domain_grads)
    number_of_domains, dim = dgr.shape

    #
    # _mean = np.mean(dgr, axis=1)[:, np.newaxis]
    # _std = np.std(dgr, axis=1)[:, np.newaxis]
    # dgr = (dgr-_mean)/_std
    dgr /= np.sqrt(np.sum(dgr**2, axis=1))[:, np.newaxis]
    
    #
    g = np.random.randn(dim)
    g /= np.sqrt(np.sum(g**2))
    u_ = np.zeros(number_of_domains)
    
    iter = 0
    # log()
    while not np.min(dgr@g/dim) >= epsilon:
        # if iter >= 100:
        #     break
        log()
        u_ = u_ + alpha*((1.+eff)*epsilon - (dgr@g)/dim)
        g = (1./number_of_domains)*np.sum(((1+u_).reshape(number_of_domains, 1))*dgr, axis=0)

        iter += 1
    log()
    return g

### Testcases

Now let's write some test cases to make sure everything is working correctly


In [29]:
epsilon=10
alpha=0.9

In [30]:
grads = 0.00000001*np.random.randn(2, 3) - 1000000
print(grads)

[[-1000000.00000001  -999999.99999999  -999999.99999999]
 [-1000000.00000001  -999999.99999999 -1000000.        ]]


In [31]:
g = gradient_maker(grads, epsilon=epsilon, alpha=alpha)
print(g)

Iter=0, Condition=-0.1908558848253521
Iter=1, Condition=3.690590098780939
Iter=2, Condition=5.883413069146656
Iter=3, Condition=7.41838914840266
Iter=4, Condition=8.492872403881861
Iter=5, Condition=9.245010682717304
Iter=6, Condition=9.771507477902114
Iter=7, Condition=10.140055234531479
[-17.56309086 -17.56309086 -17.56309086]


In [35]:
G = np.concatenate((g.reshape(1, -1), grads), axis=0)

In [48]:
A = G@G.T

In [66]:
def null(A, eps=1e-15):
    u, s, vh = np.linalg.svd(A)
    null_space = np.compress(s <= eps, vh, axis=0)
    return null_space.T

In [81]:
n = null(A)
n /= np.sum(n[1:])

In [84]:
grads@g

array([52689272.57328969, 52689272.57328983])

In [85]:
grads@(-n[0,0]*g)

array([3.e+12, 3.e+12])